## unzip and parse fit file

In [58]:
import zipfile  #zipfile.is_zipfile(filename)
import os
import fitparse
import csv
import pytz
import re

In [62]:
def parse_file(path):
    files = os.listdir(path)
    fit_files = [file for file in files if file[-4:].lower()=='.fit']
    for file in fit_files:
        new_filename = file[:-4] + '.csv'
        if os.path.exists(new_filename):
            continue
        fitfile = fitparse.FitFile(os.path.join(path,file),  
            data_processor=fitparse.StandardUnitsDataProcessor())
        if re.findall(r"WELLNESS\Z", file[:-4]):
            write_fitfile_to_csv(fitfile, os.path.join(path,new_filename))

In [63]:
# allowed_fields = ['serial_number','time_created','manufacturer',
#                   'garmin_product','number','type','timestamp[s]',
#                   'serial_number','manufacturer','garmin_product',
#                   'software_version','version','timestamp[s]',
#                   'local_timestamp[s]','cycles_to_distance[m/cycle]',
#                   'cycles_to_calories[kcal/cycle]','unknown',
#                   'resting_metabolic_rate[kcal / day]','activity_type',
#                   'timestamp[s]','unknown','timestamp[s]','enabled',
#                   'unknown','stress_level_time','stress_level_value',
#                   'unknown'
#                  ]

#final fields recorded
allowed_fields = ['timestamp_16','heart_rate','stress_level_time','stress_level_value']

#required_fields = ['serial_number', 'time_created']

UTC = pytz.UTC
CST = pytz.timezone('US/Pacific')
def write_fitfile_to_csv(fitfile, output_file='test_output.csv'):
    messages = fitfile.messages
    data = []
    for m in messages:
        skip=False
        if not hasattr(m, 'fields'):
            continue
        fields = m.fields
        #check for important data types
        mdata = {}
        for field in fields:
            if field.name in allowed_fields:
                if field.name=='stress_level_time':
                    mdata[field.name] = UTC.localize(field.value).astimezone(CST)
                else:
                    mdata[field.name] = field.value
        #for rf in required_fields:
        #    if rf not in mdata:
        #        skip=True
        if not skip:
            data.append(mdata)
    #write to csv
    with open(output_file, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(allowed_fields)
        for entry in data:
            writer.writerow([ str(entry.get(k, '')) for k in allowed_fields])
    print('wrote %s' % output_file)

In [64]:
def read_file_studio(path):
    files = os.listdir(path)
    for file in files:
        sub_file_path = os.path.join(path,file)
        
        if not os.path.isdir(sub_file_path):
            continue
        sub_files = os.listdir(sub_file_path)
        for sub_file in sub_files:
            sub_sub_file_path = os.path.join(path,file,sub_file)
            if not os.path.isdir(sub_sub_file_path):
                continue
            sub_sub_files = os.listdir(sub_sub_file_path)
            for sub_sub_file in sub_sub_files:
                
                bottom_file_path = os.path.join(path,file,sub_file, sub_sub_file)
                if not os.path.isdir(bottom_file_path):
                    continue
                bottom_files = os.listdir(bottom_file_path)
                for bottom_file in bottom_files:
                    path_to_zip_file = os.path.join(path,file,sub_file, sub_sub_file, bottom_file)
                    if zipfile.is_zipfile(path_to_zip_file):
                        print(bottom_file)
                        directory_to_extract_to = path_to_zip_file[:-4]
                        with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
                            zip_ref.extractall(directory_to_extract_to)
                        parse_file(directory_to_extract_to) # parse the fit files to csv files
        

read_file_studio("/Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio")

2022-11-13 (1).zip
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1113/2022-11-13 (1)/161902867414_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1113/2022-11-13 (1)/161902789005_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1113/2022-11-13 (1)/161811042186_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1113/2022-11-13 (1)/161870557379_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1113/2022-11-13 (1)/161902883980_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1113/2022-11-13 (1)/161900878950_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1113/2022-11-13 (1)/161863296922_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1113/2022-1

wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1111/2022-11-11/161583571624_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1111/2022-11-11/161586720746_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1111/2022-11-11/161616525665_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1111/2022-11-11/161595582604_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1111/2022-11-11/161595691668_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1111/2022-11-11/161606492071_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1111/2022-11-11/161620696298_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Danqi/1111/2022-11-11/161614396024_WELLNESS.csv
wrote /Users/lue

wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Afro/1112/2022-11-12/161737630758_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Afro/1112/2022-11-12/161737648639_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Afro/1112/2022-11-12/161749107973_WELLNESS.csv
2022-11-10.zip
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Afro/1110/2022-11-10/161463284890_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Afro/1110/2022-11-10/161452798455_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Afro/1110/2022-11-10/161446007701_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Afro/1110/2022-11-10/161486484015_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Afro/1110/2022-11-10/161465510083_WELLNESS.csv
wrote /Us

wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Helen/1112/2022-11-12 (4)/161734168309_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Helen/1112/2022-11-12 (4)/161764733565_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Helen/1112/2022-11-12 (4)/161764724854_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Helen/1112/2022-11-12 (4)/161738482245_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Helen/1112/2022-11-12 (4)/161746306504_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Helen/1112/2022-11-12 (4)/161764717182_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Helen/1112/2022-11-12 (4)/161746293482_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Helen/1112/2022-11-12 (4)/1617612416

wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Manan/1116/2022-11-16 (3)/162279978809_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Manan/1116/2022-11-16 (3)/162301137865_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Manan/1116/2022-11-16 (3)/162287254677_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Manan/1116/2022-11-16 (3)/162284886458_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Manan/1116/2022-11-16 (3)/162293953727_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Manan/1116/2022-11-16 (3)/162283299237_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Manan/1116/2022-11-16 (3)/162314151836_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Manan/1116/2022-11-16 (3)/1622799819

wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Aditya/1115/2022-11-15 (1)/162130302176_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Aditya/1115/2022-11-15 (1)/162130470455_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Aditya/1115/2022-11-15 (1)/162132356667_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Aditya/1115/2022-11-15 (1)/162163547184_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Aditya/1115/2022-11-15 (1)/162143987714_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Aditya/1115/2022-11-15 (1)/162130384653_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Aditya/1115/2022-11-15 (1)/162135850804_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/M/Aditya/1115/2022-11-15 (1)/16

In [66]:
def read_file_wsp(path):
    files = os.listdir(path)
    for file in files:
        sub_file_path = os.path.join(path,file)
        if not os.path.isdir(sub_file_path):
            continue
        sub_files = os.listdir(sub_file_path)
        for sub_file in sub_files:
            bottom_file_path = os.path.join(path,file,sub_file)
            if not os.path.isdir(bottom_file_path):
                continue
            bottom_files = os.listdir(bottom_file_path)
            for bottom_file in bottom_files:
                path_to_zip_file = os.path.join(path,file,sub_file, bottom_file)
                if zipfile.is_zipfile(path_to_zip_file):
                    directory_to_extract_to = path_to_zip_file[:-4]
                    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
                        zip_ref.extractall(directory_to_extract_to)
                    parse_file(directory_to_extract_to) # parse the fit files to csv files
read_file_wsp("/Users/luerlyu/Desktop/Experimental Data/Physiological data/WSP")

wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/WSP/Archana/1108/2022-11-08 (3)/161209672406_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/WSP/Archana/1108/2022-11-08 (3)/161160538866_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/WSP/Archana/1108/2022-11-08 (3)/161209699318_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/WSP/Archana/1108/2022-11-08 (3)/161160535580_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/WSP/Archana/1108/2022-11-08 (3)/161209675938_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/WSP/Archana/1107/Archa 2022-11-07/161117549780_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/WSP/Archana/1107/Archa 2022-11-07/161035403020_WELLNESS.csv
wrote /Users/luerlyu/Desktop/Experimental Data/Physiological data/WSP/Archana/1107/Archa 2022-11-07/161031423386_WELLNESS.csv
wr

## parse eda by specific time

In [71]:
import pandas as pd
import numpy as np
import datetime

In [119]:
def to_date(unix):
    dt = datetime.datetime.fromtimestamp(unix/1e3)
    return dt.replace(microsecond=0)

In [110]:
eda_df.head()

,Unix Timestamp (UTC),EDA (microS)
0,2022-11-17 12:34:48,0.394609
1,2022-11-17 12:34:48,0.540900
2,2022-11-17 12:34:48,0.687180
3,2022-11-17 12:34:48,0.833463
4,2022-11-17 12:34:49,0.979750


In [77]:
eda_df

,Unix Timestamp (UTC),EDA (microS)
0,2022-11-17 12:34:48.205,0.394609
1,2022-11-17 12:34:48.455,0.540900
2,2022-11-17 12:34:48.705,0.687180
3,2022-11-17 12:34:48.955,0.833463
4,2022-11-17 12:34:49.205,0.979750
...,...,...
49237,2022-11-17 15:59:57.767,74.064254
49238,2022-11-17 15:59:58.017,74.064250
49239,2022-11-17 15:59:58.267,74.064243
49240,2022-11-17 15:59:58.517,74.064250


In [120]:
eda_df.rename(columns = {"Unix Timestamp (UTC)":"time"}, inplace = True)

In [121]:
eda_df.head()

,time,EDA (microS)
0,2022-11-17 12:34:48,0.394609
1,2022-11-17 12:34:48,0.540900
2,2022-11-17 12:34:48,0.687180
3,2022-11-17 12:34:48,0.833463
4,2022-11-17 12:34:49,0.979750


In [100]:
print(eda_df["Unix Timestamp (UTC)"].iloc[-1])

2022-11-17 15:59:58.767000


In [111]:
interval_df = pd.DataFrame({"Unix Timestamp (UTC)":pd.date_range(eda_df["Unix Timestamp (UTC)"].iloc[0], eda_df["Unix Timestamp (UTC)"].iloc[-1], freq='10T')})

In [113]:
res_df = eda_df.merge(interval_df, on="Unix Timestamp (UTC)")

In [112]:
interval_df.head()

,Unix Timestamp (UTC)
0,2022-11-17 12:34:48
1,2022-11-17 12:44:48
2,2022-11-17 12:54:48
3,2022-11-17 13:04:48
4,2022-11-17 13:14:48


In [116]:
res_df.groupby(['Unix Timestamp (UTC)']).mean()

,EDA (microS)
Unix Timestamp (UTC),
2022-11-17 12:34:48,0.614038
2022-11-17 12:44:48,74.064251
2022-11-17 12:54:48,74.064244
2022-11-17 13:04:48,74.064258
2022-11-17 13:14:48,74.064249
2022-11-17 13:24:48,74.064247
2022-11-17 13:34:48,74.064256
2022-11-17 13:44:48,74.064248
2022-11-17 13:54:48,0.004958


In [138]:
def select_time_eda(path):
    eda_df = pd.read_csv(path)
    eda_df.rename(columns = {"Unix Timestamp (UTC)":"time"}, inplace = True)
    eda_df["time"] = eda_df["time"].map(to_date)
    st_time = eda_df["time"].iloc[0]
    end_time = eda_df["time"].iloc[-1]
    #make a 10 min interval
    interval_df = pd.DataFrame({"time":pd.date_range(st_time, end_time, freq='10T')})
    res_df = eda_df.merge(interval_df, on="time")
    res_df = res_df.groupby(["time"]).mean()
    path_to_new_file = path[:-4]+"_processed.csv"
    res_df.to_csv(path_to_new_file, index="False")

In [144]:
def select_time_temp(path):
    temp_df = pd.read_csv(path)
    temp_df.rename(columns = {"Unix Timestamp (UTC)":"time"}, inplace = True)
    try:
        temp_df["time"] = temp_df["time"].map(to_date)
    except:
        print(path)
        return
    
    
    res_df = temp_df.groupby(pd.Grouper(key="time", freq="10min")).mean()
    path_to_new_file = path[:-4]+"_processed.csv"
    res_df.to_csv(path_to_new_file, index="False")

In [145]:
def select_time_studio(path):
    files = os.listdir(path)
    for file in files:
        sub_file_path = os.path.join(path,file)
        
        if not os.path.isdir(sub_file_path):
            continue
        sub_files = os.listdir(sub_file_path)
        for sub_file in sub_files:
            sub_sub_file_path = os.path.join(path,file,sub_file)
            if not os.path.isdir(sub_sub_file_path):
                continue
            sub_sub_files = os.listdir(sub_sub_file_path)
            for sub_sub_file in sub_sub_files:
                
                bottom_file_path = os.path.join(path,file,sub_file, sub_sub_file)
                if not os.path.isdir(bottom_file_path):
                    continue
                bottom_files = os.listdir(bottom_file_path)
                for bottom_file in bottom_files:
                    if bottom_file == "eda.csv":
                        path_to_eda_file = os.path.join(path,file,sub_file, sub_sub_file, bottom_file)
                        select_time_eda(path_to_eda_file)
                    elif bottom_file == "temp.csv":
                        path_to_temp_file = os.path.join(path,file,sub_file, sub_sub_file, bottom_file)
                        select_time_temp(path_to_temp_file)
        

select_time_studio("/Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio")

/Users/luerlyu/Desktop/Experimental Data/Physiological data/Studio/F/Afro/1111/temp.csv


'Experimental Data/Physiological data/Studio/F/Afro/1111/temp.csv' was processed in different wa

In [146]:
def select_time_wsp(path):
    files = os.listdir(path)
    for file in files:
        sub_file_path = os.path.join(path,file)
        if not os.path.isdir(sub_file_path):
            continue
        sub_files = os.listdir(sub_file_path)
        for sub_file in sub_files:
            bottom_file_path = os.path.join(path,file,sub_file)
            if not os.path.isdir(bottom_file_path):
                continue
            bottom_files = os.listdir(bottom_file_path)
            for bottom_file in bottom_files:
                if bottom_file == "eda.csv":
                    path_to_eda_file = os.path.join(path,file,sub_file,  bottom_file)
                    select_time_eda(path_to_eda_file)
                elif bottom_file == "temp.csv":
                    path_to_temp_file = os.path.join(path,file,sub_file, bottom_file)
                    select_time_df(path_to_temp_file)
select_time_wsp("/Users/luerlyu/Desktop/Experimental Data/Physiological data/WSP")